In [2]:
import requests
import bs4
import json
import os

In [15]:
max_pages = 0
current_page = 0
typology_index = 0

path_images_thumbnails = './data/images_thumbnails/'
path_images_numpy = './data/images_numpy/'
path_annotations_json = './data/annotations_json/'
path_annotations_pickle = './data/annotations_pickle/'

url_base = 'https://collections.louvre.fr/en/recherche?page=1&typology%5B0%5D=1'
url_scrap = f'https://collections.louvre.fr/en/recherche?page={current_page}&typology[{typology_index}]=1'
url_art_DB_base = 'https://collections.louvre.fr/en/ark:/53355/'



#typology = {'Peintures' : f'Typology[{typology_index}]=1'}

# DOM content
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
# https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent

In [52]:
def get_weblimits(url):

    response = requests.get(url, headers=headers) #, timeout=0.2)
    html = response.content
    print(f'Status code is: {response.status_code}')

    response = requests.get(url, headers=headers) #, timeout=0.05)
    html = response.content
    soup = bs4.BeautifulSoup(html, "html.parser")

    span = soup.find('input', class_='pagination__current nav__pagination__nbr no_style')
    max_number_of_pages = int(span['max'])
    current_page = int(span['value'])

    print(f'There are {max_number_of_pages} pages to be scraped.\n')

    return max_number_of_pages, current_page

In [56]:
def get_artworks_arkId(url):

    arkId_list = []

    response = requests.get(url, headers=headers) #, timeout=0.05)
    print(f'Status code is: {response.status_code}')
    html = response.content
    soup = bs4.BeautifulSoup(html, "html.parser")

    span = soup.findAll('a', class_='card--search card__link')

    for i in range(0, len(span)):
        #print(span[i]['href'][-11:])
        arkId_list.append(span[i]['href'][-11:])

    print(f'There are {len(arkId_list)} items in this page.\n')    
    return arkId_list

In [57]:
# Get item thumbnail and metadata
def get_images_and_metadata(arkId_list):

    for dir in [path_images_thumbnails, path_annotations_json]:

        if os.path.exists(dir):
            pass
        else:
            os.mkdir(dir)

    # Get item info
    for arkId in arkId_list:
        metadata = requests.get(url_art_DB_base+ arkId +'.json').content
        print(f'./json/{arkId[-11:]}')

        with open(f'{path_annotations_json}{arkId[-11:]}.json', 'wb') as f:
            f.write(metadata) 


        thumbnail = requests.get(json.loads(metadata)['image'][0]['urlThumbnail']).content
        print(json.loads(metadata)['image'][0]['urlThumbnail'])

        with open(f'{path_images_thumbnails}{arkId[-11:]}.jpg', 'wb') as f:
            f.write(thumbnail)

In [59]:
max_pages, current_page = get_weblimits(url_base)
arkId_list = []

# 10 webs / 1.5 min

for i in range(current_page,3):
    url_scrap = f'https://collections.louvre.fr/en/recherche?page={i}&typology[{typology_index}]=1'
    print(f'Scanning page {i} of {max_pages}')
    #print(url_scrap, '\n')

    arkId_list += get_artworks_arkId(url_scrap)
    get_images_and_metadata(arkId_list)

Status code is: 200
There are 511 pages to be scraped.

Scanning page 1 of 511
Status code is: 200
There are 20 items in this page.

./json/cl010416624
https://collections.louvre.fr/media/cache/small/0000000021/0000416624/0000981034_OG.JPG
./json/cl010416494
https://collections.louvre.fr/media/cache/small/0000000021/0000416494/0000887843_OG.JPG
./json/cl010428225
https://collections.louvre.fr/media/cache/small/0000000021/0000428225/0000800467_OG.JPG
./json/cl010426221
https://collections.louvre.fr/media/cache/small/0000000021/0000426221/0000800468_OG.JPG
./json/cl010427862
https://collections.louvre.fr/media/cache/small/0000000021/0000427862/0000846211_OG.JPG
./json/cl010251069
https://collections.louvre.fr/media/cache/small/0000000021/0000251069/0000911628_OG.JPG
./json/cl010251071
https://collections.louvre.fr/media/cache/small/0000000021/0000251071/0000911631_OG.JPG
./json/cl010251078
https://collections.louvre.fr/media/cache/small/0000000021/0000251078/0000911625_OG.JPG
./json/cl01